In [3]:
setIndex_1=0

In [4]:
os.getcwd()
os.chdir(projectDir)

In [5]:
setIndex_2= setIndex_1+10
print(setIndex_2)

10


In [7]:
adataC = sc.read('outputs/anndata/adata-hash-features_singlets_SingleKO_08122020_PerGENE.h5ad')
allGeneNames = pd.read_csv("AllExpGeneNames.csv", index_col=0)


In [8]:
adataC.X.mean(axis=0).shape

(6685,)

In [9]:
covariates = adataC.uns['feature_barcode_names_filtered_GENES']
covariates = covariates[covariates != "GENE_CONTROL_"]

In [10]:
len(covariates)

1031

In [11]:
guideMatrix = adataC.obs[covariates]
guideMatrix.shape
guideMatrix.rename(columns = {'GENE_Rnf8-cmtr1_':'GENE_Rnf8cmtr1_'}, inplace = True) 
guideMatrix.rename(columns = {'GENE_Siah1-ps1_':'GENE_Siah1ps1_'}, inplace = True) 
guideMatrix.rename(columns = {'GENE_Siah1-ps2_':'GENE_Siah1ps2_'}, inplace = True) 

KOGenes = pd.Series(guideMatrix.columns)

In [12]:
## Add covariates that should be corrected for
df = adataC.obs[["n_genes", "mt_frac", "leiden" ]]

guideMatrix = guideMatrix.join(df)
guideMatrix.shape

(242938, 1034)

In [13]:
expressionMatrix = pd.DataFrame(adataC.raw.X.A)
expressionMatrix.columns = allGeneNames.iloc[:,0]
expressionMatrix = expressionMatrix[adataC.var_names]
expressionMatrix.index = adataC.obs.index

In [14]:
selColumns = pd.Series(KOGenes[setIndex_1:setIndex_2]).append(pd.Series(["n_genes", "mt_frac", "leiden"]))

In [15]:
sum(adataC.obs["GENE_CONTROL_"] == 1)

41123

In [18]:
k = adataC[adataC.obs["GENE_CONTROL_"] == 1]
k.obs["leiden"].value_counts()

0    12120
1     9730
2     7980
3     6149
4     1603
5     1259
6      954
7      701
8      584
9       43
Name: leiden, dtype: int64

In [21]:
k2 = k[0:10000,]
k2.obs["leiden"].value_counts()

0    3171
1    2627
2    2147
3    1201
4     287
6     254
7     151
5     118
8      43
9       1
Name: leiden, dtype: int64

In [13]:
controlExpressionMat = expressionMatrix.loc[adataC.obs["GENE_CONTROL_"] == 1,]
controlGuideMat = guideMatrix.loc[adataC.obs["GENE_CONTROL_"] == 1,]
controlExpressionMat = controlExpressionMat.iloc[0:5000,:]
controlGuideMat = controlGuideMat.iloc[0:5000,:]
controlGuideMat=controlGuideMat[selColumns]

In [14]:
geneExpressionMat = expressionMatrix.loc[adataC.obs[covariates[setIndex_1:setIndex_2]].sum(axis=1) > 0,]
geneGuideMat = guideMatrix.loc[adataC.obs[covariates[setIndex_1:setIndex_2]].sum(axis=1) > 0,]
geneGuideMat=geneGuideMat[selColumns]

In [15]:
allExpMat = np.concatenate((controlExpressionMat, geneExpressionMat))
allGuideMat = controlGuideMat.append(geneGuideMat)

In [16]:
allVars = pd.Series(KOGenes[setIndex_1:setIndex_2]) 
allVars

0    GENE_4921501E09Rik_
1    GENE_4930595M18Rik_
2    GENE_8030462N17Rik_
3             GENE_Aamp_
4            GENE_Abtb1_
5            GENE_Abtb2_
6            GENE_Acaca_
7             GENE_Ago2_
8           GENE_Ahctf1_
9              GENE_Ahr_
dtype: object

In [17]:
all_columns = "+".join(allVars[~allVars.isin(["leiden"])])
all_columns = all_columns +  "+n_genes+mt_frac"

In [18]:
my_formula = "y~" + all_columns + " + (1 | leiden)"
my_formula

'y~GENE_4921501E09Rik_+GENE_4930595M18Rik_+GENE_8030462N17Rik_+GENE_Aamp_+GENE_Abtb1_+GENE_Abtb2_+GENE_Acaca_+GENE_Ago2_+GENE_Ahctf1_+GENE_Ahr_+n_genes+mt_frac + (1 | leiden)'

In [19]:
allResp=adataC.var_names
allResp

Index(['Mrpl15', 'Lypla1', 'Tcea1', 'Atp6v1h', 'Rb1cc1', 'Pcmtd1', 'Rrs1',
       'Vcpip1', 'Sgk3', 'Snhg6',
       ...
       'Cacul1', 'Eif3a', 'Fam45a', 'Prdx3', 'Grk5', 'Zfp950', 'Csf2ra',
       'Vamp7', 'AC149090.1', 'CAAA01118383.1'],
      dtype='object', length=6685)

In [20]:
allGuideMat

,GENE_4921501E09Rik_,GENE_4930595M18Rik_,GENE_8030462N17Rik_,GENE_Aamp_,GENE_Abtb1_,GENE_Abtb2_,GENE_Acaca_,GENE_Ago2_,GENE_Ahctf1_,GENE_Ahr_,n_genes,mt_frac,leiden
AAACGCTAGTTGTAGA-1-E3-1,0,0,0,0,0,0,0,0,0,0,951,0.086056,0
AAAGGATTCAGACAAA-1-E3-1,0,0,0,0,0,0,0,0,0,0,1247,0.071198,2
AAAGGGCAGACGGTTG-1-E3-1,0,0,0,0,0,0,0,0,0,0,1514,0.073052,1
AAAGGGCAGAGATCGC-1-E3-1,0,0,0,0,0,0,0,0,0,0,1561,0.062370,0
AAAGGGCGTTGATGTC-1-E3-1,0,0,0,0,0,0,0,0,0,0,1095,0.051056,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
TGCGATAGTGTTCGTA-1-E4-24,0,0,0,0,0,1,0,0,0,0,1769,0.056831,0
TTAGGGTGTGAGTTTC-1-E4-24,0,0,0,0,0,0,0,0,1,0,2340,0.076101,2
TTCATTGTCTGAACGT-1-E4-24,1,0,0,0,0,0,0,0,0,0,1945,0.059586,2
TTCTGTAAGTGTAGTA-1-E4-24,0,0,0,0,0,0,0,1,0,0,1636,0.099162,1


In [ ]:
%%R -i allGuideMat,allExpMat,my_formula,allResp,setIndex_1,setIndex_2  -w 8 -h 4 -u in
library("MASS")
library("lme4")

for(i in seq(1,ncol(allExpMat),20)){
    print(i)
    
    coefDF = data.frame()
    for(j in seq(i,i+20,1)){
       
        
      tryCatch(
            expr = {
                   allGuideMat["y"] = allExpMat[,j]
                   myFit <- glmer.nb(formula(my_formula), data = allGuideMat, control = glmerControl(optimizer="bobyqa", calc.derivs = FALSE))
                   k <- summary(myFit)
                   myDF <- data.frame(coefficients(k))
                   #colnames(myRes) <- dimnames(coefficients(k))[[2]]
                   myDF$coefNames <- dimnames(coefficients(k))[[1]]
                   myDF$respGene = allResp[j]
                   print(myDF)
                   coefDF <- rbind(coefDF, myDF)
            },
            error = function(e){ 
                print("111111111111")
                print(e)
            },
            finally = {
                # (Optional)
                # Do this at the end before quitting the tryCatch structure...
            }
      )
     
    }
    
     saveRDS(coefDF, paste0("./MixedEffectNegativeBinomialLMOutputs/coefs_",setIndex_1,"_",setIndex_2,"_", i, "_",i+20,".rds"))
    

}

R[write to console]: Loading required package: Matrix



[1] 1
                         Estimate   Std..Error     z.value     Pr...z..
(Intercept)         -2.3488561993 1.596295e-01 -14.7144231 5.208856e-49
GENE_4921501E09Rik_  0.2625180074 1.222803e-01   2.1468545 3.180487e-02
GENE_4930595M18Rik_  0.0397279527 1.124780e-01   0.3532065 7.239337e-01
GENE_8030462N17Rik_ -0.2537923460 2.960597e-01  -0.8572338 3.913157e-01
GENE_Aamp_           0.1726078241 2.367193e-01   0.7291666 4.658998e-01
GENE_Abtb1_          0.0245510007 1.593904e-01   0.1540306 8.775856e-01
GENE_Abtb2_          0.1471089950 1.475720e-01   0.9968624 3.188313e-01
GENE_Acaca_          0.2632842614 1.285024e-01   2.0488666 4.047516e-02
GENE_Ago2_           0.0358911968 1.592742e-01   0.2253422 8.217131e-01
GENE_Ahctf1_        -0.1834976242 1.397312e-01  -1.3132188 1.891092e-01
GENE_Ahr_           -0.2441091126 2.081843e-01  -1.1725627 2.409712e-01
n_genes              0.0004726026 7.368584e-05   6.4137505 1.419825e-10
mt_frac              2.0292250417 1.733217e+00   1.1707854

Exception ignored from cffi callback <function _processevents at 0x7fe84ca5b8b0>:
Traceback (most recent call last):
  File "/home/beraslan/miniconda3/lib/python3.8/site-packages/rpy2/rinterface_lib/callbacks.py", line 277, in _processevents
    try:
KeyboardInterrupt: 
